In [59]:
current = pd.read_csv('current_raw.csv')
current.columns = [c.replace('localtime', 'local_time') for c in current.columns]
columns = current.columns

In [61]:
current

,query,name,region,country,lat,lon,tz_id,local_time_epoch,local_time,last_updated_epoch,...,precip_in,humidity,cloud,feelslike_c,feelslike_f,vis_km,vis_miles,uv,gust_mph,gust_kph
0,"FORTALEZA, CEARA, BRAZIL",Fortaleza,Ceara,Brazil,-3.32,-41.42,America/Fortaleza,1703791929,2023-12-28 16:32,1703791800,...,0.0,71,50,35.1,95.1,10.0,6.0,6.0,20.6,33.1


In [54]:
print(f"CREATE TABLE IF NOT EXISTS current_weather_raw ({' text,'.join(columns)} text)")

CREATE TABLE IF NOT EXISTS current_weather_raw (query text,name text,region text,country text,lat text,lon text,tz_id text,time_local_epoch text,time_local text,last_updated_epoch text,last_updated text,temp_c text,temp_f text,is_day text,condition text,wind_mph text,wind_kph text,wind_degree text,wind_dir text,pressure_mb text,pressure_in text,precip_mm text,precip_in text,humidity text,cloud text,feelslike_c text,feelslike_f text,vis_km text,vis_miles text,uv text,gust_mph text,gust_kph text)


In [58]:
current = pd.read_csv('./current_raw.csv')
current['update_time'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
current.columns = [c.replace('localtime', 'local_time') for c in current.columns]
columns = current.columns
conn = psycopg2.connect("dbname='postgres' user='root' host='172.19.0.1' password='root'")
cur = conn.cursor()

cur.execute("DROP TABLE IF EXISTS current_weather_raw")
cur.execute(f"CREATE TABLE IF NOT EXISTS current_weather_raw ({' text,'.join(columns)} text)")

for index, row in current.iterrows():
    cur.execute(f"INSERT INTO current_weather_raw ({','.join(columns)}) VALUES ({','.join(['%s']*len(columns))})", row)
    
conn.commit()
cur.close()
conn.close()


In [51]:
# replace `localtime` with time_local in columns
current.columns = [c.replace('localtime', 'time_local') for c in current.columns]

INSERT INTO current_weather_raw (name,region,country,lat) VALUES (%s,%s,%s,%s) query                                          FORTALEZA, CEARA, BRAZIL
name                                                          Fortaleza
region                                                            Ceara
country                                                          Brazil
lat                                                               -3.32
lon                                                              -41.42
tz_id                                                 America/Fortaleza
localtime_epoch                                              1703791272
localtime                                              2023-12-28 16:21
last_updated_epoch                                           1703790900
last_updated                                           2023-12-28 16:15
temp_c                                                             31.0
temp_f                                                   

In [4]:
-- postgresql database
SELECT uuid as uuid,
cast(local_time as timestamp) as local_time,
cast( last_updated as timestamp) as last_updated,
--define timezone  for local time
cast(local_time as timestamp) at time zone 'UTC' at time zone tz_id as local_time,
-- define utc time for last updated
cast(last_updated as timestamp) at time zone 'UTC' as last_updated,
query as location,
country as country,
region as state,
name as city,
cast(lat as float) as latitude,
cast(lon as float) as longitude,
tz_id as timezone,
cast(temp_c as float) as temperture_c,
cast(feelslike_c as float) as feelslike_c,
cast(wind_kph as float) as wind_kph,
cast(wind_degree as float) as wind_angle,
wind_dir as wind_direction,
cast(pressure_in as float) as pressure_in,
cast(precip_mm as float) as precipitation_mm,
cast(humidity as float) as humidity,
cast(cloud as float) as cloud_cover,
cast(vis_km as float) as visibility_km,
cast(uv as float) as uv_index
FROM public.current_weather_raw
LIMIT 100

In [8]:
import psycopg2
import pandas as pd

conn = psycopg2.connect("dbname='postgres' user='root' host='172.19.0.1' password='root'")
cur = conn.cursor()

cur.execute(f"SELECT * FROM current_weather_raw")
columns = [desc[0] for desc in cur.description]
rows = cur.fetchall()
cur.close()
conn.close()
current = pd.DataFrame(rows, columns=columns)
current

,uuid,query,name,region,country,lat,lon,tz_id,local_time_epoch,local_time,...,humidity,cloud,feelslike_c,feelslike_f,vis_km,vis_miles,uv,gust_mph,gust_kph,update_time
0,f25db075-3a69-4abf-87ca-c73a99c9dc86,"MANAUS, AMAZONAS, BRAZIL",Manaus,Amazonas,Brazil,-3.11,-60.03,America/Manaus,1703793534,2023-12-28 15:58,...,71,50,36.0,96.8,10.0,6.0,7.0,4.7,7.5,2023-12-28 19:59:30
1,bf45dedb-5cd4-4fac-b1dd-fadefcebb400,"FORTALEZA, CEARA, BRAZIL",Fortaleza,Ceara,Brazil,-3.32,-41.42,America/Fortaleza,1703793535,2023-12-28 16:58,...,71,50,35.1,95.1,10.0,6.0,6.0,20.6,33.1,2023-12-28 19:59:30
2,2bc1537f-f708-4b5c-b409-d8a0133b18f3,"BELO HORIZONTE, MINAS GERAIS, BRAZIL",Belo Horizonte,Minas Gerais,Brazil,-19.92,-43.93,America/Sao_Paulo,1703793537,2023-12-28 16:58,...,62,50,32.0,89.5,10.0,6.0,5.0,11.4,18.4,2023-12-28 19:59:30
3,796aa48b-8cec-4fab-9c78-cacbb14ea12b,"MACEIO, ALAGOAS, BRAZIL",Maceio,Alagoas,Brazil,-9.67,-35.72,America/Maceio,1703793547,2023-12-28 16:59,...,66,25,32.9,91.1,10.0,6.0,7.0,13.8,22.2,2023-12-28 19:59:30
4,0f126bcd-bc72-4f5f-a17c-a7bb2bcb218f,"TERESINA, PIAUI, BRAZIL",Teresina,Piaui,Brazil,-5.08,-42.82,America/Fortaleza,1703793539,2023-12-28 16:58,...,59,75,35.7,96.3,10.0,6.0,8.0,12.5,20.2,2023-12-28 19:59:30
5,f562ae5a-4af0-42e7-9a1a-9954ced01893,"RECIFE, PERNAMBUCO, BRAZIL",Recife,Pernambuco,Brazil,-8.05,-34.9,America/Recife,1703793563,2023-12-28 16:59,...,70,50,31.9,89.4,10.0,6.0,7.0,10.4,16.7,2023-12-28 19:59:30
6,40ea4181-81f7-42fe-9d9d-ef364cd26924,"SALVADOR, BAHIA, BRAZIL",Salvador,Bahia,Brazil,-12.98,-38.52,America/Bahia,1703793534,2023-12-28 16:58,...,66,25,35.5,95.8,10.0,6.0,7.0,19.7,31.7,2023-12-28 19:59:30
7,e3fb2aac-6f3c-466f-b46d-003e53d561e0,"ARACAJU, SERGIPE, BRAZIL",Aracaju,Sergipe,Brazil,-10.92,-37.07,America/Maceio,1703793545,2023-12-28 16:59,...,70,25,32.8,91.0,10.0,6.0,6.0,19.5,31.3,2023-12-28 19:59:30
8,a8ad2fb0-d525-4462-a7ef-1af20116cf19,"PALMAS, TOCANTINS, BRAZIL",Palmas,Tocantins,Brazil,-10.22,-48.28,America/Araguaina,1703793545,2023-12-28 16:59,...,100,75,26.3,79.3,6.0,3.0,6.0,11.5,18.5,2023-12-28 19:59:30
9,8075a2a4-37a1-4063-8d7a-c8ebb9328fa6,"SAO LUIS, MARANHAO, BRAZIL",Sao Luis,Maranhao,Brazil,-2.52,-44.27,America/Fortaleza,1703793537,2023-12-28 16:58,...,71,75,40.4,104.8,10.0,6.0,6.0,20.9,33.7,2023-12-28 19:59:30


In [23]:
import json

In [62]:
# replace single quotes with double quotes
forecast['forecastday'] = forecast['forecastday'].apply(lambda x: x.replace("'", '"'))
forecast['forecastday'] = forecast['forecastday'].apply(lambda x: json.loads(x))
forecast = forecast.explode('forecastday')
forecast['forecast_date'] = forecast['forecastday'].apply(lambda x: x['date'])
forecast['forecast_day_info'] = forecast['forecastday'].apply(lambda x: x['day'])
forecast['forecast_day_info'].apply(lambda x:  json.dumps(x))

forecast['forecast_day_maxtemp_c'] = forecast['forecast_day_info'].apply(lambda x: x['maxtemp_c'])
forecast['forecast_day_mintemp_c'] = forecast['forecast_day_info'].apply(lambda x: x['mintemp_c'])
forecast['forecast_day_avgtemp_c'] = forecast['forecast_day_info'].apply(lambda x: x['avgtemp_c'])
forecast['forecast_day_maxwind_kph'] = forecast['forecast_day_info'].apply(lambda x: x['maxwind_kph'])
forecast['forecast_day_totalprecip_mm'] = forecast['forecast_day_info'].apply(lambda x: x['totalprecip_mm'])
forecast['forecast_day_avgvis_km'] = forecast['forecast_day_info'].apply(lambda x: x['avgvis_km'])
forecast['forecast_day_avghumidity'] = forecast['forecast_day_info'].apply(lambda x: x['avghumidity'])
forecast['forecast_day_daily_will_it_rain'] = forecast['forecast_day_info'].apply(lambda x: x['daily_will_it_rain'])
forecast['forecast_day_daily_chance_of_rain'] = forecast['forecast_day_info'].apply(lambda x: x['daily_chance_of_rain'])
forecast['forecast_day_daily_will_it_snow'] = forecast['forecast_day_info'].apply(lambda x: x['daily_will_it_snow'])
forecast['forecast_day_daily_chance_of_snow'] = forecast['forecast_day_info'].apply(lambda x: x['daily_chance_of_snow'])
forecast['forecast_day_condition_text'] = forecast['forecast_day_info'].apply(lambda x: x['condition']['text'])
forecast['forecast_day_condition_icon'] = forecast['forecast_day_info'].apply(lambda x: x['condition']['icon'])
forecast['forecast_day_condition_code'] = forecast['forecast_day_info'].apply(lambda x: x['condition']['code'])
forecast['forecast_day_uv'] = forecast['forecast_day_info'].apply(lambda x: x['uv'])
forecast['forecast_day_hour'] = forecast['forecastday'].apply(lambda x: x['hour'])

forecast_day_columns = ['forecast_date', 'forecast_day_maxtemp_c', 'forecast_day_mintemp_c', 'forecast_day_avgtemp_c', 'forecast_day_maxwind_kph', 'forecast_day_totalprecip_mm', 'forecast_day_avgvis_km', 'forecast_day_avghumidity', 'forecast_day_daily_will_it_rain', 'forecast_day_daily_chance_of_rain', 'forecast_day_daily_will_it_snow', 'forecast_day_daily_chance_of_snow', 'forecast_day_condition_text', 'forecast_day_condition_icon', 'forecast_day_condition_code', 'forecast_day_uv', 'forecast_day_hour']
location_columns = ['name', 'region', 'country', 'lat', 'lon', 'tz_id']
forecast['forecast_day_hour'] = forecast['forecast_day_hour'].apply(lambda x: str(x).replace("'", '"'))

forecast = forecast[location_columns + forecast_day_columns]
forecast


,name,region,country,lat,lon,tz_id,forecast_date,forecast_day_maxtemp_c,forecast_day_mintemp_c,forecast_day_avgtemp_c,...,forecast_day_avghumidity,forecast_day_daily_will_it_rain,forecast_day_daily_chance_of_rain,forecast_day_daily_will_it_snow,forecast_day_daily_chance_of_snow,forecast_day_condition_text,forecast_day_condition_icon,forecast_day_condition_code,forecast_day_uv,forecast_day_hour
0,Manaus,Amazonas,Brazil,-3.11,-60.03,America/Manaus,2023-12-28,32.4,24.5,27.2,...,81.0,1,84,0,0,Patchy rain possible,//cdn.weatherapi.com/weather/64x64/day/176.png,1063,6.0,"[{""time_epoch"": 1703736000, ""time"": ""2023-12-2..."
0,Manaus,Amazonas,Brazil,-3.11,-60.03,America/Manaus,2023-12-29,32.5,23.8,27.5,...,80.0,1,89,0,0,Moderate rain,//cdn.weatherapi.com/weather/64x64/day/302.png,1189,6.0,"[{""time_epoch"": 1703822400, ""time"": ""2023-12-2..."
0,Manaus,Amazonas,Brazil,-3.11,-60.03,America/Manaus,2023-12-30,28.7,23.6,25.2,...,92.0,1,89,0,0,Moderate rain,//cdn.weatherapi.com/weather/64x64/day/302.png,1189,6.0,"[{""time_epoch"": 1703908800, ""time"": ""2023-12-3..."
